In [64]:
import numpy as np
import sys
import math
import torch
from torch import nn
def hilbertCurve(order):
    for i in range(0,order):
        if i == 0:
            curve = np.array([[0,1],[3,2]])
        else:
            offset = 4**(i)
            q1 = np.rot90(np.flip(curve,0),-1)
            q2 = curve + offset
            q3 = curve + offset*2
            q4 = np.rot90(np.flip(curve,1),-1) + offset*3
            top = np.concatenate((q1,q2),axis=1)
            bot = np.concatenate((q4,q3),axis=1)
            curve = np.concatenate((top,bot))
    return(curve)

def Get_Hilbert_index(order,sequential):
    the_curve = hilbertCurve(order)
    index = np.argwhere(the_curve == sequential)
    return index

In [65]:

####将张量转化为Hilber展开
def From_Tensor_to_Line(input_tensor):
    device = input_tensor.device()
    input_shape = input_tensor.shape
    the_order = int(np.log2(input_tensor.shape[3]))
    padding_method = nn.ConstantPad2d((1,1,1,1),0)####填充
    input_tensor = padding_method(input_tensor)

    output_tensor = torch.rand([input_shape[0],input_shape[1],3,input_shape[2]*input_shape[2]*3])
    
    for i in range(input_shape[2]*input_shape[2]):
        index = Get_Hilbert_index(order=the_order,sequential=i)
        index1 = index[0][0]
        index2 = index[0][1] 
        for p in [-1,0,1]:
            for q in [-1,0,1]:
                output_tensor[:,:,1+p,3*i+1+q] = input_tensor[:,:,index1+1+p,index2+1+q]
    output_tensor = output_tensor.to(device)            

    return output_tensor




In [66]:


def From_Line_to_2d(input_tensor):
    device = input_tensor.device()
    input_shape = input_tensor.shape
    size = int(math.sqrt(input_shape[3]))
    the_order = int(np.log2(size))

    output_tensor = torch.rand([input_shape[0],input_shape[1],size,size])
    for i in range(input_shape[3]):
        index = Get_Hilbert_index(order=the_order,sequential=i)
        index1 = index[0][0]
        index2 = index[0][1]
        output_tensor[:,:,index1,index2] = input_tensor[:,:,0,i]
    output_tensor = output_tensor.to(device)  
    return output_tensor



    